### Step - Install Jupytab

https://towardsdatascience.com/interactive-simulation-with-tableau-and-jupytab-c26adb1be564
<br>
https://github.com/CFMTech/Jupytab#configuration-file

#### Jupytab has two components, jupytab and jupytab-server, that need to be installed in their own environments:
<br>
- jupytab: a very simple API that exposes dataframes and custom functions from a notebook. It needs be installed in the notebook environment, and only requires Pandas as a dependency.
<br>
<br>
- jupytab-server: provides the Web Data Connector, spawns kernels, manages the configuration and so on… Do not install this component in the same environment as your notebook, so as to prevent conflicts.

#### 1. Environment 1 - jupytab notebook environment

- 1. We first create a virtual environment using Python to keep everything clean and reproducible
<br>
conda create -n jupytab-notebook-env python=3.9.6
<br>
conda activate jupytab-notebook-env
<br>
<br>
- 2. Then we install the latest jupytab version and the ipykernel library so as to make our Jupyter kernel available in notebooks:
<br>
conda install jupytab=0.9.11
<br>
<br>
- 3. Then we install the new kernel in the new Jupyter environment:
<br>
conda install ipykernel
<br>
python -m ipykernel install --user --name jupytab-notebook-env --display-name "jupytab-outlook-python-tablea"
<br>
<br>
#### here "--name" is the name of the env we want to install the kernel to, then "--display-name" is the kernel name we want to see inside jupyter notebook
<br>
<br>
- 4. inside the terminal, install jupyter then launch jupyter notebook:
<br>
conda install jupyter
<br>
jupyter notebook

Test environment

In [1]:
!conda info


     active environment : base
    active env location : /Users/magidbugazia/anaconda3
            shell level : 2
       user config file : /Users/magidbugazia/.condarc
 populated config files : /Users/magidbugazia/.condarc
          conda version : 4.8.3
    conda-build version : 3.18.9
         python version : 3.7.4.final.0
       virtual packages : __osx=10.16
       base environment : /Users/magidbugazia/anaconda3  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/osx-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/osx-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://conda.anaconda.org/conda-forge/osx-64
                          https://conda.anaconda.org/conda-forge/noarch
          package cache : /Users/magidbugazia/anaconda3/pkgs
                          /Users/magidbugazia/.conda/pkgs
       envs directories : /Use

check that Jupytab has been properly installed

In [2]:
import jupytab
print(jupytab.__version__)

0.9.11


#### 2. Environment 2 - server side

- 1. ideactivate current env
<br>
conda deactivate
<br>
<br>
- 2. create new environment for the server
<br>
conda create -n jupytab-server-env python
<br>
<br>
- 3. install jupytab-server
<br>
conda install jupytab-server
<br>
<br>
- 4. For notebook kernels, you must install the jupytab library that only have a dependency on Pandas.
<br>
conda install jupytab

### Step -  Jupyter Notebook - connect to Outlook and Setup Parameters

Find env 

In [2]:
import os
print(os.getcwd())
print(os.__file__)

/Users/magidbugazia/Documents/Super Petroleum/Codes/Python
/Users/magidbugazia/anaconda3/envs/jupytab-notebook-env/lib/python3.9/os.py


In [4]:
pwd

'/Users/magidbugazia/Documents/Super Petroleum/Codes/Python'

Set up libraries

xchangelin is the library that allows to read a file-like object in-memory instead of pandas that read data frame you have to pre-download

#### first install exchangelib
in terminal do pip install exchangelib

#### but, since we are using a conda environment. download inside the environment:make sure it is inside the jupytab notebook env

conda install exchangelib

##### this might give an error that:

PackagesNotFoundError: The following packages are not available from current channels:

  - exchangelib

Current channels:

(it will print out a list of urrent channels)
  
#### to fix this, first display  list of active channels inside conda by typing either one of the following commands:
conda info
<br> or 
conda config --show channels

make sure you have a forge channel in the list, if not we can easily add one
<br> 
#### we add a forge channel, conda-forge channel to your list of channels with this command:
conda config --append channels conda-forge

#### It tells conda to also look on the conda-forge channel when you search for packages. You can then simply install the two packages with:

conda install exchangelib

#### sources: 
exchange lib official doc:  https://pypi.org/project/exchangelib/
<br>
exchange lib official doc:  https://ecederstrand.github.io/exchangelib/#installation
<br>
fix exchange lib env issue: https://stackoverflow.com/questions/48493505/packagesnotfounderror-the-following-packages-are-not-available-from-current-cha
<br>
important reference discussions: https://stackoverflow.com/questions/36133721/python-download-excel-file-from-email-attachment-then-parse-it
<br>
important reference discussions: https://stackoverflow.com/questions/53950601/create-dataframe-from-excel-attachment-in-outlook

In [5]:
import io

from exchangelib import DELEGATE, Account, Credentials, Configuration, FileAttachment, ItemAttachment, Message, \
  CalendarItem, HTMLBody
import pandas as pd

credentials = Credentials('magid@superpetroleum.com', '100ml33FL21*')
config = Configuration(server='outlook.office365.com', credentials=credentials)
account = Account(
    primary_smtp_address='magid@superpetroleum.com',
    config=config,
    autodiscover=False,
    access_type=DELEGATE
)

### Step - Fetch the attachemnet from the specific email

look up the last 7 emails

In [6]:
for item in account.inbox.all().order_by('-datetime_received')[:10]:
 print(item.subject, item.sender, item.datetime_received)

Transfer Holbrook to return Mailbox(name='Mike Palermo', email_address='mpalermo@superpetroleum.com', routing_type='SMTP', mailbox_type='Mailbox') 2021-08-23 13:01:17+00:00
[SuperPetroleum - Administrator]Report : SKU Sales Detail By Sub Category/Station Mailbox(name='autoreply@s2konline.net', email_address='autoreply@s2konline.net', routing_type='SMTP', mailbox_type='OneOff') 2021-08-23 07:19:00+00:00
[SuperPetroleum - Administrator]Report : SKU Sales Detail By Sub Category/Station Mailbox(name='autoreply@s2konline.net', email_address='autoreply@s2konline.net', routing_type='SMTP', mailbox_type='OneOff') 2021-08-23 04:29:55+00:00
Transfer slips Mailbox(name='Roslindale Station', email_address='roslindalestation@gmail.com', routing_type='SMTP', mailbox_type='OneOff') 2021-08-21 13:36:44+00:00
RE: Masks Sales Report Mailbox(name='Daisy Liu', email_address='DLiu@SuperPetroleum.com', routing_type='SMTP', mailbox_type='Mailbox') 2021-08-20 20:45:39+00:00
FW: Masks Sales Report Mailbox(name

1. Find the items in the inbox matching the email subject you specify

In [9]:
item = account.inbox.all().get(subject='[SuperPetroleum - Administrator]Report : SKU Sales Detail By Sub Category/Station')

- you need to work on the code above:
    <br>
    right now, when it finds two or more attachments with the same name it give an error, because it is not sure which one it should pick
    <br>
    How to make it iterate starting from the top, then stop when it finds the first match (the latest report)??

#### - Iterate through the attachments and match with the filename you specify
2. The attachment content will be the excel file in the form of a byte string

In [10]:
for attachment in item.attachments:
    if attachment.name == 'SKU Detail By Sub Category-Station.xls':
        my_excel_file_in_bytes = attachment.content
        break
else:
    assert False, 'No attachment with that name'

3. Now that you have the excel file in bytes, convert to a file-like object and read the excel file in memory

#### in terminal inside our environment

conda install xlrd

In [100]:
my_excel_file_io = io.BytesIO(my_excel_file_in_bytes)
df = pd.read_excel(io=my_excel_file_io)

WARNING *** file size (2564032) not 512 + multiple of sector size (512)


### Step - Call our Dataframe, clea, and preprocess it

In [101]:
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,SuperPetroleum**,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/23/2021 12:18:42 AM (Pacific Standard Time),NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Detailed SKU Report By Sub Category/Station,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,30 Station(s) From: 6/24/2021 To 8/22/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,SKU Number,NaN,Description,NaN,# of SalesDays,Total Sold,Avg / Day,Cost / Item,Sell / Item,Total Cost,NaN,Total Sell,NaN,Profit%
8,NaN,01-Braintree 1 [TC] #17701,NaN,NaN,NaN,NaN,NaN,NaN,3.252213,5.71658,46617.4143,NaN,81941.6098,NaN,0.431091
9,NaN,Sub Category: BEVERAGE,NaN,NaN,NaN,NaN,2780,NaN,1.259875,1.925131,3502.4861,NaN,5351.9202,NaN,0.345564


get rid of extra rows (first 6)

In [102]:
df = df.iloc[7: , :]
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
7,NaN,SKU Number,NaN,Description,NaN,# of SalesDays,Total Sold,Avg / Day,Cost / Item,Sell / Item,Total Cost,NaN,Total Sell,NaN,Profit%
8,NaN,01-Braintree 1 [TC] #17701,NaN,NaN,NaN,NaN,NaN,NaN,3.252213,5.71658,46617.4143,NaN,81941.6098,NaN,0.431091
9,NaN,Sub Category: BEVERAGE,NaN,NaN,NaN,NaN,2780,NaN,1.259875,1.925131,3502.4861,NaN,5351.9202,NaN,0.345564
10,NaN,1200000113,NaN,Pepsi 2Liter,NaN,20,24,1.2,1.3563,1.99,32.5534,NaN,47.76,NaN,0.3183
11,NaN,1200000129,NaN,PEPSI Cola 20oz,NaN,33,57,1.727273,1.1917,1.8721,67.9269,NaN,106.71,NaN,0.3634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,NaN,Sub Category: UNUSED,NaN,NaN,NaN,NaN,1,NaN,0,9.97,0,NaN,9.97,NaN,1
9144,NaN,9997,NaN,ITEM D,NaN,1,1,1,0,9.97,0,NaN,9.97,NaN,1
9145,NaN,Sub Category: CANDY,NaN,NaN,NaN,NaN,1,NaN,0,20,0,NaN,20,NaN,1
9146,NaN,2000,NaN,UNKNOWN,NaN,1,1,1,0,20,0,NaN,20,NaN,1


make first row a header

In [103]:
df.columns = df.iloc[0] #rab the first row as the df header
df = df[1:] #take the data less the header row

In [112]:
df.head(10)

7,NaN,SKU Number,NaN,Description,NaN,# of SalesDays,Total Sold,Avg / Day,Cost / Item,Sell / Item,Total Cost,NaN,Total Sell,NaN,Profit%
8,NaN,01-Braintree 1 [TC] #17701,NaN,NaN,NaN,NaN,NaN,NaN,3.252213,5.71658,46617.4143,NaN,81941.6098,NaN,0.431091
9,NaN,Sub Category: BEVERAGE,NaN,NaN,NaN,NaN,2780,NaN,1.259875,1.925131,3502.4861,NaN,5351.9202,NaN,0.345564
10,NaN,1200000113,NaN,Pepsi 2Liter,NaN,20,24,1.2,1.3563,1.99,32.5534,NaN,47.76,NaN,0.3183
11,NaN,1200000129,NaN,PEPSI Cola 20oz,NaN,33,57,1.727273,1.1917,1.8721,67.9269,NaN,106.71,NaN,0.3634
12,NaN,1200000131,NaN,MOUNTAIN DEW 20 OZ,NaN,43,103,2.395349,1.1917,1.8432,122.7451,NaN,189.85,NaN,0.3534
13,NaN,1200000134,NaN,MOUNTAIN DEW DIET 20 OZ,NaN,18,20,1.111111,1.1034,1.942,22.068,NaN,38.84,NaN,0.4318
14,NaN,1200000159,NaN,Aquafina 20z Water,NaN,30,92,3.066667,0.5417,0.99,49.8413,NaN,91.08,NaN,0.4527
15,NaN,1200000180,NaN,STARBUCKS FRAPP COFFEE 13.7OZ,NaN,3,5,1.666667,2.06,2.99,10.3,NaN,14.95,NaN,0.311
16,NaN,1200000233,NaN,MOUNTAIN DEW 2LTR,NaN,15,27,1.8,1.4226,1.99,38.4102,NaN,53.73,NaN,0.2851
17,NaN,1200000294,NaN,PEPSI 1 LTR,NaN,9,10,1.111111,1.34,2.19,13.4,NaN,21.9,NaN,0.3881


#### Create new column 'station name' and populate it with corresponding station name from other column

- First, since so far the data is impotred, clean, and ready for manipulations, it's good practice to copy it into a new data frame so that if we miss up we can come back to this point.
<br>
- think of it as a check point for our data.
<br>
- In case we needed to strat over, instead of going all the way back to df, we just get back to this check point

In [133]:
Sales_Data = df.copy() ## you need to add .copy() otherwise any change to data_sales will also accure to df

In [134]:
Sales_Data.head(10)

7,NaN,SKU Number,NaN,Description,NaN,# of SalesDays,Total Sold,Avg / Day,Cost / Item,Sell / Item,Total Cost,NaN,Total Sell,NaN,Profit%
8,NaN,01-Braintree 1 [TC] #17701,NaN,NaN,NaN,NaN,NaN,NaN,3.252213,5.71658,46617.4143,NaN,81941.6098,NaN,0.431091
9,NaN,Sub Category: BEVERAGE,NaN,NaN,NaN,NaN,2780,NaN,1.259875,1.925131,3502.4861,NaN,5351.9202,NaN,0.345564
10,NaN,1200000113,NaN,Pepsi 2Liter,NaN,20,24,1.2,1.3563,1.99,32.5534,NaN,47.76,NaN,0.3183
11,NaN,1200000129,NaN,PEPSI Cola 20oz,NaN,33,57,1.727273,1.1917,1.8721,67.9269,NaN,106.71,NaN,0.3634
12,NaN,1200000131,NaN,MOUNTAIN DEW 20 OZ,NaN,43,103,2.395349,1.1917,1.8432,122.7451,NaN,189.85,NaN,0.3534
13,NaN,1200000134,NaN,MOUNTAIN DEW DIET 20 OZ,NaN,18,20,1.111111,1.1034,1.942,22.068,NaN,38.84,NaN,0.4318
14,NaN,1200000159,NaN,Aquafina 20z Water,NaN,30,92,3.066667,0.5417,0.99,49.8413,NaN,91.08,NaN,0.4527
15,NaN,1200000180,NaN,STARBUCKS FRAPP COFFEE 13.7OZ,NaN,3,5,1.666667,2.06,2.99,10.3,NaN,14.95,NaN,0.311
16,NaN,1200000233,NaN,MOUNTAIN DEW 2LTR,NaN,15,27,1.8,1.4226,1.99,38.4102,NaN,53.73,NaN,0.2851
17,NaN,1200000294,NaN,PEPSI 1 LTR,NaN,9,10,1.111111,1.34,2.19,13.4,NaN,21.9,NaN,0.3881


Check Which rows contain a specific station name

In [135]:
Sales_Data[Sales_Data['SKU Number'].str.contains("Holbrook")]

7,NaN,SKU Number,NaN,Description,NaN,# of SalesDays,Total Sold,Avg / Day,Cost / Item,Sell / Item,Total Cost,NaN,Total Sell,NaN,Profit%
5666,NaN,17-Holbrook [TC] #17717,NaN,NaN,NaN,NaN,NaN,NaN,2.412896,4.583749,77133.4657,NaN,146529.1044,NaN,0.473598


#### Problem - previous function is long if we wanted to apply to all stations and looks like bad practice

1. maybe create a list called station_names, then create a function that reads if row value in station names return row value

In [136]:
stations_names = ["Braintree 1", "Braintree 2", "Hingham", "Lawrence", 
                   "Salem NH", "Quincy Sea", "Saugus", "Holbrook",
                   "Stoneham", "Methuen", "Roslindale", "Milton", 
                   "North Andover", "Kingston", "Waltham", "Bourne"]

In [137]:
import numpy as np
Sales_Data['Station Name'] = np.where(Sales_Data['SKU Number'].str.contains('|'.join(stations_names)), Sales_Data['SKU Number'], float('NaN'))

The previous code reads as follwos:

- np.where: takes three arguments, like an ifelse in R: e.g. ifelse(Sales$`SKU Number` %in% station_names, Sales$`SKU Number`, Station)
- First argument: df['SKU Number'].str.contains('|'.join(stations_names): if SKU Number column contains string present in stations_names list, then put second argument
- Second arguments: df['SKU Number']: use that string present in the column because we match the condition, otherwise put third argument
- Third Argument: float('NaN'): if condition was not met attach a NaN value

In [138]:
Sales_Data

7,NaN,SKU Number,NaN,Description,NaN,# of SalesDays,Total Sold,Avg / Day,Cost / Item,Sell / Item,Total Cost,NaN,Total Sell,NaN,Profit%,Station Name
8,NaN,01-Braintree 1 [TC] #17701,NaN,NaN,NaN,NaN,NaN,NaN,3.252213,5.71658,46617.4143,NaN,81941.6098,NaN,0.431091,01-Braintree 1 [TC] #17701
9,NaN,Sub Category: BEVERAGE,NaN,NaN,NaN,NaN,2780,NaN,1.259875,1.925131,3502.4861,NaN,5351.9202,NaN,0.345564,NaN
10,NaN,1200000113,NaN,Pepsi 2Liter,NaN,20,24,1.2,1.3563,1.99,32.5534,NaN,47.76,NaN,0.3183,NaN
11,NaN,1200000129,NaN,PEPSI Cola 20oz,NaN,33,57,1.727273,1.1917,1.8721,67.9269,NaN,106.71,NaN,0.3634,NaN
12,NaN,1200000131,NaN,MOUNTAIN DEW 20 OZ,NaN,43,103,2.395349,1.1917,1.8432,122.7451,NaN,189.85,NaN,0.3534,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,NaN,Sub Category: UNUSED,NaN,NaN,NaN,NaN,1,NaN,0,9.97,0,NaN,9.97,NaN,1,NaN
9144,NaN,9997,NaN,ITEM D,NaN,1,1,1,0,9.97,0,NaN,9.97,NaN,1,NaN
9145,NaN,Sub Category: CANDY,NaN,NaN,NaN,NaN,1,NaN,0,20,0,NaN,20,NaN,1,NaN
9146,NaN,2000,NaN,UNKNOWN,NaN,1,1,1,0,20,0,NaN,20,NaN,1,NaN


check to see if all stations came through

In [139]:
Sales_Data['Station Name'].unique()

array(['01-Braintree 1 [TC] #17701', nan, '02-Hingham [TC] #17702',
       '06-Lawrence [TC] #17706', '18-Saugus [TC] #17718',
       '36-Roslindale [TC] #27770', '03-Braintree 2 [TC] #17703',
       '07-Salem NH [R] NOT TAXABLE #17707', '09-Quincy Sea [TC] #17709',
       '17-Holbrook [TC] #17717', '31-Kingston [TC] #17763',
       '37- Waltham [P] #27772', '38-Granite Ave Milton [P] #27773',
       '30-Methuen [RK] #17762'], dtype=object)

2.Create new column 'Department' and populate it with corresponding departments from other column

In [141]:
department_names = ["AUTOMOTIVE", "BEVERAGE", "BOTTLE DEP", "C-Store",
                      "CANDY", "CIGARETTES", "FOUNTAIN", "DAIRY", "GROCERY",  
                     "HBA", "ICE CREAM","NEWS/MAGAZINE", "SCRATCHERS",
                     "SNACKS", "UNUSED", "TOBACCO"]

In [142]:
Sales_Data['Department'] = np.where(Sales_Data['SKU Number'].str.contains('|'.join(department_names)), Sales_Data['SKU Number'], float('NaN'))

In [143]:
Sales_Data

7,NaN,SKU Number,NaN,Description,NaN,# of SalesDays,Total Sold,Avg / Day,Cost / Item,Sell / Item,Total Cost,NaN,Total Sell,NaN,Profit%,Station Name,Department
8,NaN,01-Braintree 1 [TC] #17701,NaN,NaN,NaN,NaN,NaN,NaN,3.252213,5.71658,46617.4143,NaN,81941.6098,NaN,0.431091,01-Braintree 1 [TC] #17701,NaN
9,NaN,Sub Category: BEVERAGE,NaN,NaN,NaN,NaN,2780,NaN,1.259875,1.925131,3502.4861,NaN,5351.9202,NaN,0.345564,NaN,Sub Category: BEVERAGE
10,NaN,1200000113,NaN,Pepsi 2Liter,NaN,20,24,1.2,1.3563,1.99,32.5534,NaN,47.76,NaN,0.3183,NaN,NaN
11,NaN,1200000129,NaN,PEPSI Cola 20oz,NaN,33,57,1.727273,1.1917,1.8721,67.9269,NaN,106.71,NaN,0.3634,NaN,NaN
12,NaN,1200000131,NaN,MOUNTAIN DEW 20 OZ,NaN,43,103,2.395349,1.1917,1.8432,122.7451,NaN,189.85,NaN,0.3534,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,NaN,Sub Category: UNUSED,NaN,NaN,NaN,NaN,1,NaN,0,9.97,0,NaN,9.97,NaN,1,NaN,Sub Category: UNUSED
9144,NaN,9997,NaN,ITEM D,NaN,1,1,1,0,9.97,0,NaN,9.97,NaN,1,NaN,NaN
9145,NaN,Sub Category: CANDY,NaN,NaN,NaN,NaN,1,NaN,0,20,0,NaN,20,NaN,1,NaN,Sub Category: CANDY
9146,NaN,2000,NaN,UNKNOWN,NaN,1,1,1,0,20,0,NaN,20,NaN,1,NaN,NaN


####  Popoulate empty cells in both 'Station' and Department Columns

In [144]:
Sales_Data['Station Name'] = Sales_Data['Station Name'].fillna(method='ffill')

In [145]:
Sales_Data['Department'] = Sales_Data['Department'].fillna(method='ffill')

In [146]:
Sales_Data

7,NaN,SKU Number,NaN,Description,NaN,# of SalesDays,Total Sold,Avg / Day,Cost / Item,Sell / Item,Total Cost,NaN,Total Sell,NaN,Profit%,Station Name,Department
8,NaN,01-Braintree 1 [TC] #17701,NaN,NaN,NaN,NaN,NaN,NaN,3.252213,5.71658,46617.4143,NaN,81941.6098,NaN,0.431091,01-Braintree 1 [TC] #17701,NaN
9,NaN,Sub Category: BEVERAGE,NaN,NaN,NaN,NaN,2780,NaN,1.259875,1.925131,3502.4861,NaN,5351.9202,NaN,0.345564,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
10,NaN,1200000113,NaN,Pepsi 2Liter,NaN,20,24,1.2,1.3563,1.99,32.5534,NaN,47.76,NaN,0.3183,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
11,NaN,1200000129,NaN,PEPSI Cola 20oz,NaN,33,57,1.727273,1.1917,1.8721,67.9269,NaN,106.71,NaN,0.3634,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
12,NaN,1200000131,NaN,MOUNTAIN DEW 20 OZ,NaN,43,103,2.395349,1.1917,1.8432,122.7451,NaN,189.85,NaN,0.3534,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,NaN,Sub Category: UNUSED,NaN,NaN,NaN,NaN,1,NaN,0,9.97,0,NaN,9.97,NaN,1,30-Methuen [RK] #17762,Sub Category: UNUSED
9144,NaN,9997,NaN,ITEM D,NaN,1,1,1,0,9.97,0,NaN,9.97,NaN,1,30-Methuen [RK] #17762,Sub Category: UNUSED
9145,NaN,Sub Category: CANDY,NaN,NaN,NaN,NaN,1,NaN,0,20,0,NaN,20,NaN,1,30-Methuen [RK] #17762,Sub Category: CANDY
9146,NaN,2000,NaN,UNKNOWN,NaN,1,1,1,0,20,0,NaN,20,NaN,1,30-Methuen [RK] #17762,Sub Category: CANDY


#### Clean dataframe

Get Columns names

In [147]:
for col in Sales_Data:
    print(col)

nan
SKU Number
nan
Description
nan
# of SalesDays
Total Sold
Avg / Day
Cost / Item
Sell / Item
Total Cost
nan
Total Sell
nan
Profit%
Station Name
Department


Drop unwanted columns

In [148]:
Sales_Data = Sales_Data.drop(['# of SalesDays', 'Avg / Day', 'Total Cost', 'Total Sell', 'Profit%'], axis = 1)

Drop the columns where all elements are NaN:

In [149]:
Sales_Data = Sales_Data.dropna(axis=1, how='all')

drop rows with NA values

In [150]:
Sales_Data = Sales_Data.dropna()

In [153]:
Sales_Data

7,SKU Number,Description,Total Sold,Cost / Item,Sell / Item,Station Name,Department
10,1200000113,Pepsi 2Liter,24,1.3563,1.99,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
11,1200000129,PEPSI Cola 20oz,57,1.1917,1.8721,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
12,1200000131,MOUNTAIN DEW 20 OZ,103,1.1917,1.8432,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
13,1200000134,MOUNTAIN DEW DIET 20 OZ,20,1.1034,1.942,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
14,1200000159,Aquafina 20z Water,92,0.5417,0.99,01-Braintree 1 [TC] #17701,Sub Category: BEVERAGE
...,...,...,...,...,...,...,...
9138,69872021326,FULL of $100,1,1.2009,2,38-Granite Ave Milton [P] #27773,Sub Category: SCRATCHERS
9139,69872021327,CASH TO GO!,62,1.8558,5,38-Granite Ave Milton [P] #27773,Sub Category: SCRATCHERS
9142,1000,THANK YOU BAGS,1,0,10,30-Methuen [RK] #17762,Sub Category: SCRATCHERS
9144,9997,ITEM D,1,0,9.97,30-Methuen [RK] #17762,Sub Category: UNUSED


Inspect dataframe

In [155]:
Sales_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8933 entries, 10 to 9146
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SKU Number    8933 non-null   object
 1   Description   8933 non-null   object
 2   Total Sold    8933 non-null   object
 3   Cost / Item   8933 non-null   object
 4   Sell / Item   8933 non-null   object
 5   Station Name  8933 non-null   object
 6   Department    8933 non-null   object
dtypes: object(7)
memory usage: 558.3+ KB


how many items sold per station

In [156]:
Sales_Data['Station Name'].value_counts()

17-Holbrook [TC] #17717               1587
36-Roslindale [TC] #27770             1171
02-Hingham [TC] #17702                1158
07-Salem NH [R] NOT TAXABLE #17707    1005
01-Braintree 1 [TC] #17701             755
38-Granite Ave Milton [P] #27773       692
37- Waltham [P] #27772                 684
06-Lawrence [TC] #17706                493
18-Saugus [TC] #17718                  491
31-Kingston [TC] #17763                438
03-Braintree 2 [TC] #17703             325
09-Quincy Sea [TC] #17709              131
30-Methuen [RK] #17762                   3
Name: Station Name, dtype: int64

### Step -  Connect to Tableau Server

#### Tables definition
<br>
The publication of data sources for Tableau from a notebook is done through two classes:

1. Tables: Contains the publication-ready tables provided by the notebook. There is typically a single instance of this class in a given notebook.
<br>
2. DataFrameTable: Table for either static or dynamic publication in Tableau. Static tables never change on the Tableau side. Dynamic tables are regenerated for each Tableau Extract.
<br>
<br>
For referrence: https://github.com/CFMTech/Jupytab#configuration-file

Load the dataframe in a Jupytab Tables dictionary, to indicate that we want to expose those data to Tableau:

In [159]:
tables = jupytab.Tables()

In [161]:
tables['Sales_Data'] = jupytab.DataFrameTable("Sales_Data", dataframe=Sales_Data, include_index=True)

In [162]:
tables

#### Expose tables schema:- In order to allow jupytab-server to retrieve data, we need to add two cells.
1. The first cell generates a schema that declares all our Dataframes. It needs to be exactly like the cell below

In [163]:
# GET /schema
tables.render_schema()

[{"id": "sales_data", "alias": "Sales_Data", "columns": [{"id": "index", "dataType": "int"}, {"id": "SKU_Number", "dataType": "string"}, {"id": "Description", "dataType": "string"}, {"id": "Total_Sold", "dataType": "string"}, {"id": "Cost_Item", "dataType": "string"}, {"id": "Sell_Item", "dataType": "string"}, {"id": "Station_Name", "dataType": "string"}, {"id": "Department", "dataType": "string"}]}]


2. The second cell is where data is exported:

Executing this cell generates a harmless error in the notebook: the REQUEST variable is only available when the notebook is executed by the Jupytab server:

conda install flask

In [164]:
# GET /data
tables.render_data(REQUEST)

NameError: name 'REQUEST' is not defined

That was a harmless error, REQUEST will only be defined when running with Jupytab, so the error is harmless.)
<br>
If you don’t want to get the error during execution, you can wrap render_data() with a simple try block:

In [165]:
# GET /data
try:
    tables.render_data(REQUEST)
except NameError:
    print("Not available outside jupytab context")

Not available outside jupytab context


### Configure and Launch the Jupytab server
<br>
In the Jupytab server environment, we need to create a configuration file that will allow us to configure a few parameters like the server port, a secret token and of course the list of notebooks that the server must exposes.
<br>
<br>
 The config.ini file tells Jupytab which notebooks contain the tables that should be published for Tableau

- Write a config.ini file in text ediior. Put it anywhere you want. preferrably in the same environment you're working in

[main]
<br>
listen_port = 8123
<br>
#you can add a securtiy token 
<br>
#my_token = Goldberg4*
<br>
notebooks = SalesData_Outlook_Python_Tableau
 <br>
 <br>
[SalesData_Outlook_Python_Tableau]
<br>
path = /Users/magidbugazia/anaconda3/envs/jupytab-notebook-env/SalesData_Outlook_Python_Tableau.ipynb
<br>
description = Monthly Sales Data transferred from S2K to Jupyter Notebook to Tableau


#### in terminal:
<br>
- In the jupytab server environment, Launch the jubytab

jupytab --config=config.ini 
<br>
<br>
OR
<br>
<br>
jupytab --config=/Users/magidbugazia/anaconda3/envs/jupytab-server-env/config.ini

#### The ouput contains two important pieces of information:
<br>
1. The list of published notebooks.
<br>
2. The URL to be used in Tableau in order to access the data (including any security token declared in the configuration file).

#### note: this migh give a jupytab command not found in terminal

- this is because the intended command you’re attempting to use is located in a nonstandard directory or in another location (/usr/local/sbin/ etc)
<br>
<br>
- to fix this, first print out the current paths in use:
<br>
(jupytab-server-env) echo $PATH
<br>
<br>
- add the a new directory (where jupytab exists) to our path lits using:
<br>

(jupytab-server-env) export PATH=$PATH:/Users/magidbugazia/anaconda3/envs/jupytab-server-env/conda-meta

#### this is the data web connctor urk needed for tableau
<br>
- Please open : http://bluejaziacs-Mac.fios-router.home:8123

In [ ]:
/Documents/00 - Career/Online Courses/AWS-SAACO2-Maark/